In [21]:
import pandas as pd
import numpy as np 
import string
#!pip install keras
import tensorflow as tf
import random
import pickle

In [3]:
dwarves = open("Dwarf Names Training Data.txt","r")
dwarf_names = []
for dwarf in dwarves:
    dwarf_stripped = dwarf.rstrip()
    dwarf_stripped = dwarf_stripped.lower()
    dwarf_names.append(dwarf_stripped)

dwarf_names = list(map(lambda s: s + '.', dwarf_names))
    
print(dwarf_names)

['may.', 'adaldrida.', 'celendine.', 'gloriana.', 'pimpernel.', 'tanta.', 'alfrida.', 'cora.', 'goldilocks.', 'melba.', 'poppy.', 'yolanda.', 'amalda.', 'cornelia.', 'grimalda.', 'melilot.', 'prima.', 'amaranth.', 'chica.', 'hilda.', 'melissa.', 'primrose.', 'amethyst.', 'daisy.', 'jasmine.', 'menegilda.', 'primula.', 'angelica.', 'diamanda.', 'jemima.', 'mentha.', 'prisca.', 'arabella.', 'diamond.', 'jessamine.', 'mimosa.', 'regina.', 'asphodel.', 'dina.', 'lalia.', 'mirabella.', 'rhoda.', 'belba.', 'donnamira.', 'laura.', 'miranda.', 'robinia.', 'belinda.', 'dora.', 'lavinia.', 'myrtle.', 'rosa.', 'bellisima.', 'duenna.', 'lily.', 'nina.', 'rosamunda.', 'bell.', 'eglantine.', 'linda.', 'nora.', 'rose.', 'belladonna.', 'elanor.', 'lobelia.', 'pamphila.', 'rowan.', 'bertha.', 'esmeralda.', 'malva.', 'pandora.', 'ruby.', 'berylla.', 'estella.', 'marigold.', 'pansy.', 'salvia.', 'camelia.', 'fatima.', 'mantissa.', 'pearl.', 'savanna.', 'cara.', 'gerda.', 'matilda.', 'peony.', 'selina.', 

In [4]:
chars = string.ascii_lowercase
chars_num = len(chars)
print(chars_num)
print(chars)

26
abcdefghijklmnopqrstuvwxyz


In [5]:
max_length = len(max(dwarf_names, key=len))
dwarf_length = len(dwarf_names)
print(max_length)
print(dwarf_length)

11
280


In [6]:
max_length = len(max(dwarf_names, key=len))
dwarf_length = len(dwarf_names)
char_to_id = {c:i for i, c in enumerate(chars)}
char_to_id['.'] = 26

id_to_char = {v:k for v,k in enumerate(chars)}
id_to_char[26] = '.'


char_dim = len(char_to_id)
print(char_to_id)
print(id_to_char)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, '.': 26}
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z', 26: '.'}


In [7]:
X = np.zeros((dwarf_length,max_length,char_dim))
print(len(X[0][0]))

27


In [8]:
X = np.zeros((dwarf_length,max_length,char_dim))
Y = np.zeros((dwarf_length,max_length,char_dim))


In [9]:
for i in range(dwarf_length):
    dwarf = list(dwarf_names[i])
    for j in range(len(dwarf)):
        X[i,j,char_to_id[dwarf[j]]] = 1
        if j < len(dwarf)-1:
            Y[i,j,char_to_id[dwarf[j+1]]] = 1


In [10]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import LambdaCallback

In [11]:

model = Sequential()
model.add(LSTM(128, input_shape=(max_length, char_dim), return_sequences=True))
model.add(Dense(char_dim, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
def make_name(model,user_string):
    name = []
    x = np.zeros((1, max_length, char_dim))
    end = False
    i = 0
    any_length = random.randint(5,10)
    name.append(user_string)
    
    while end==False:
        probs = list(model.predict(x)[0,i])
        probs = probs / np.sum(probs)
        index = np.random.choice(range(char_dim), p=probs)
        if i == max_length-2:
            character = '.'
            end = True
        else:
            character = id_to_char[index]
        name.append(character)
        x[0, i+1, index] = 1
        i += 1
        if character == '.':
            end = True
    
    print(''.join(name))

In [13]:
def generate_name_loop(epoch, _):
    if epoch % 500 == 0:
        
        print('Names generated after epoch %d:' % epoch)

        for i in range(3):
            make_name(model,"")
        
        print()

In [14]:
name_generator = LambdaCallback(on_epoch_end = generate_name_loop)

model.fit(X, Y, batch_size=64, epochs=3000, callbacks=[name_generator], verbose=0)

# why is the character of the same length everytime? - this was because I did not have a full stop before

Names generated after epoch 0:
feiwglnao.
litaeekrb.
bmuwzhazj.

Names generated after epoch 500:
wifadm.
fseraaald.
rimzaa.

Names generated after epoch 1000:
edipakr.
osodro.
doy.

Names generated after epoch 1500:
ilarn.
ol.
ul.

Names generated after epoch 2000:
olocls.
aravrs.
ilanabopd.

Names generated after epoch 2500:
yhocma.
orl.
isba.



In [17]:
make_name(model, "Sree")

Sreexentadero.


In [24]:
model.save('my_model.h5')

In [26]:
from keras.models import load_model
loadingmodel = load_model('my_model.h5')
make_name(loadingmodel,"Suj")

Sujotu.
